In [1]:
import os
import sys
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import figure_setup as fs

# You need SKA somewhere: https://github.com/bcarry/ska
sys.path.append("/home/bcarry/Documents/softs/")
from ska import tools as skatools

# Get ZTF fit

In [2]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [3]:
# ZTF filters
filters = {"1": "g", "2": "r"}
fink_colors = ["#15284F", "#F5622E"]

In [4]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [5]:
bft = pd.read_parquet(bft_file)

In [6]:
bft.columns[bft.columns.str.contains(r"^(?=.*Atlas)(?=.*orange)")]

Index(['phase_functions.Misc/Atlas.orange.H.value',
       'phase_functions.Misc/Atlas.orange.H.error.min',
       'phase_functions.Misc/Atlas.orange.H.error.max',
       'phase_functions.Misc/Atlas.orange.G1.value',
       'phase_functions.Misc/Atlas.orange.G1.error.min',
       'phase_functions.Misc/Atlas.orange.G1.error.max',
       'phase_functions.Misc/Atlas.orange.G2.value',
       'phase_functions.Misc/Atlas.orange.G2.error.min',
       'phase_functions.Misc/Atlas.orange.G2.error.max',
       'phase_functions.Misc/Atlas.orange.N',
       'phase_functions.Misc/Atlas.orange.phase.min',
       'phase_functions.Misc/Atlas.orange.phase.max',
       'phase_functions.Misc/Atlas.orange.rms',
       'phase_functions.Misc/Atlas.orange.facility',
       'phase_functions.Misc/Atlas.orange.technique',
       'phase_functions.Misc/Atlas.orange.name_filter'],
      dtype='object')

In [7]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "phase_functions.Misc/Atlas.orange.H.value",
    "phase_functions.Misc/Atlas.orange.G1.value",
    "phase_functions.Misc/Atlas.orange.G2.value",
    "phase_functions.Misc/Atlas.cyan.H.value",
    "phase_functions.Misc/Atlas.cyan.G1.value",
    "phase_functions.Misc/Atlas.cyan.G2.value",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]

data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [8]:
data.columns[data.columns.str.contains("Atlas")]

Index(['phase_functions.Misc/Atlas.orange.H.value',
       'phase_functions.Misc/Atlas.orange.G1.value',
       'phase_functions.Misc/Atlas.orange.G2.value',
       'phase_functions.Misc/Atlas.cyan.H.value',
       'phase_functions.Misc/Atlas.cyan.G1.value',
       'phase_functions.Misc/Atlas.cyan.G2.value'],
      dtype='object')

In [9]:
len(data)

115147

# Compare results

In [10]:
data.HG1G2_status.value_counts()

HG1G2_status
 2.0    106453
 4.0      6708
 1.0      1248
 3.0       497
-2.0       238
 0.0         3
Name: count, dtype: int64

In [11]:
thres = 1e-3

# HG Parameeter
mask_HG_g = (data.HG_H_g.notna()) & (data.HG_G_g.notna())
mask_HG_r = (data.HG_H_r.notna()) & (data.HG_G_r.notna())
mask_HG_fit = (data.HG_fit == 0) & (data.HG_status >= 2)
mask_HG = mask_HG_g & mask_HG_r & mask_HG_fit

# HG1G2 parameters
mask_HG1G2_g = (
    (data.HG1G2_G1_g > thres)
    & (data.HG1G2_G2_g > thres)
    & ((1 - data.HG1G2_G1_g - data.HG1G2_G2_g) > thres)
)
mask_HG1G2_r = (
    (data.HG1G2_G1_r > thres)
    & (data.HG1G2_G2_r > thres)
    & ((1 - data.HG1G2_G1_r - data.HG1G2_G2_r) > thres)
)
mask_HG1G2_fit = (data.HG1G2_fit == 0) & (data.HG1G2_status >= 2)
mask_HG1G2 = mask_HG1G2_fit & mask_HG1G2_g & mask_HG1G2_r

# SHG1G2 ZTF
mask_sHG1G2_g = (
    (data.sHG1G2_G1_g > thres)
    & (data.sHG1G2_G2_g > thres)
    & ((1 - data.sHG1G2_G1_g - data.sHG1G2_G2_g) > thres)
)
mask_sHG1G2_r = (
    (data.sHG1G2_G1_r > thres)
    & (data.sHG1G2_G2_r > thres)
    & ((1 - data.sHG1G2_G1_r - data.sHG1G2_G2_r) > thres)
)
mask_sHG1G2_ZTF = mask_sHG1G2_g & mask_sHG1G2_r
mask_sHG1G2_fit = (data.sHG1G2_fit == 0) & (data.sHG1G2_status >= 2)
mask_sHG1G2 = mask_sHG1G2_fit & mask_sHG1G2_ZTF

# Oblateness
mask_R = (data.sHG1G2_R>0.3)

# Spin solution suspicous: RA=={0,180,360}, DEC==0
maskSpin = (
    (data.sHG1G2_alpha0 > thres)
    & (np.abs(360 - data.sHG1G2_alpha0) > thres)
    & (np.abs(data.sHG1G2_alpha0 - 180) > thres)
    & (np.abs(data.sHG1G2_delta0) > thres)
)

# FINK Sample
maskFINK = mask_sHG1G2 & mask_R & maskSpin

# Phase coverage
maskPhase = data.minphase < 2.5

# Global mask
mask = mask_HG1G2 & mask_sHG1G2


print(f" All data       : {len(data):6d}  ({100:>6.2f}%)")
print()
print(
    f"  Mask HG g      : {len(data[mask_HG_g]):6d}  ({100.*len(data[mask_HG_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG r      : {len(data[mask_HG_r]):6d}  ({100.*len(data[mask_HG_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG g+r    : {len(data[mask_HG]):6d}  ({100.*len(data[mask_HG])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask HG1G2 g   : {len(data[mask_HG1G2_g]):6d}  ({100.*len(data[mask_HG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 r   : {len(data[mask_HG1G2_r]):6d}  ({100.*len(data[mask_HG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 g+r : {len(data[mask_HG1G2]):6d}  ({100.*len(data[mask_HG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask SHG1G2 g  : {len(data[mask_sHG1G2_g]):6d}  ({100.*len(data[mask_sHG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 r  : {len(data[mask_sHG1G2_r]):6d}  ({100.*len(data[mask_sHG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 g+r: {len(data[mask_sHG1G2]):6d}  ({100.*len(data[mask_sHG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask Oblateness: {len(data[mask_R]):6d}  ({100.*len(data[mask_R])/len(data):>6.2f}%)"
)
print(
    f"  Mask Spin      : {len(data[maskSpin]):6d}  ({100.*len(data[maskSpin])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask FINK      : {len(data[maskFINK]):6d}  ({100.*len(data[maskFINK])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask (both)    : {len(data[mask]):6d}  ({100.*len(data[mask])/len(data):>6.2f}%)"
)
for minphase in [2, 3, 4, 5]:
    maskPhase = data.minphase < minphase
    print(
        f"  Mask phase {minphase}  : {len(data[maskPhase]):6d}  ({100.*len(data[maskPhase])/len(data):>6.2f}%)"
    )


# len(data), len(data[mask]), len(data[mask_HG1G2]), len(data[mask_HG1G2sp]), len(data[mask_sHG1G2_g]), len(data[mask_sHG1G2_r]), len(data[mask_sHG1G2])

 All data       : 115147  (100.00%)

  Mask HG g      : 114904  ( 99.79%)
  Mask HG r      : 114906  ( 99.79%)
  Mask HG g+r    : 112564  ( 97.76%)

  Mask HG1G2 g   :  56815  ( 49.34%)
  Mask HG1G2 r   :  62043  ( 53.88%)
  Mask HG1G2 g+r :  40727  ( 35.37%)

  Mask SHG1G2 g  :  70736  ( 61.43%)
  Mask SHG1G2 r  :  75544  ( 65.61%)
  Mask SHG1G2 g+r:  55579  ( 48.27%)

  Mask Oblateness:  80317  ( 69.75%)
  Mask Spin      : 111357  ( 96.71%)

  Mask FINK      :  42531  ( 36.94%)

  Mask (both)    :  34691  ( 30.13%)
  Mask phase 2  :  57515  ( 49.95%)
  Mask phase 3  :  79455  ( 69.00%)
  Mask phase 4  :  93564  ( 81.26%)
  Mask phase 5  : 101933  ( 88.52%)


## Difference in absolute magnitude: 2x2

In [ ]:
# VERSION 2x2 for ATLAS
fig, ax = plt.subplots(
    2,
    2,
    figsize=fs.figsize(0.5),
    sharex=True,
    sharey=True,
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.97,
        "bottom": 0.13,
        "left": 0.1,
        "right": 0.98,
    },
)

r = [-1.8,1.8]
b = 75

# --------------------------------------------------------------------------------
# Histograms
for i, f in enumerate(["g", "r"]):
    
    x = data[f"HG_H_{f}"] - data[f"sHG1G2_H_{f}"]
    ax[i, 0].hist(
        x[maskFINK],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

    x = data[f"HG1G2_H_{f}"] - data[f"sHG1G2_H_{f}"]
    ax[i, 1].hist(
        x[maskFINK & mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

# --------------------------------------------------------------------------------
# Axes
for a in ax.ravel():
    a.set_yscale("log")

ax[0, 0].set_yticks([1, 10, 1e2, 1e3, 1e4])
ax[0, 0].set_yticklabels(["", "$10^1$", "$10^2$", "$10^3$", "$10^4$"])

ax[0, 0].set_ylabel("Count in g")
ax[1, 0].set_ylabel("Count in r")


ax[1, 0].set_xlabel(
    "$\Delta$H ($\\texttt{HG}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)"
)
ax[1, 1].set_xlabel(
    "$\Delta$H ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)"
)



# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.png"),
    facecolor="white",
)

fig.savefig(
   os.path.join(data_fink, "gfx", "article", "abs_mag.pgf"),
)
plt.close()

## Difference in absolute magnitude: 3x2

In [ ]:
# VERSION 3x2 for ATLAS
fig, ax = plt.subplots(
    2,
    3,
    figsize=fs.figsize(0.5),
    sharex=True,
    sharey=True,
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.94,
        "bottom": 0.12,
        "left": 0.1,
        "right": 0.98,
    },
)

r = [-2, 2]
b = 80

# --------------------------------------------------------------------------------
# Histograms
for i, f in enumerate(["g", "r"]):
    x = data[f"HG_H_{f}"] - data[f"sHG1G2_H_{f}"]

    ax[i, 0].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

    x = data[f"HG1G2_H_{f}"] - data[f"sHG1G2_H_{f}"]
    ax[i, 1].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

    x = data[f"phase_functions.Misc/Atlas.cyan.H.value"] - data[f"sHG1G2_H_{f}"]
    ax[i, 2].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        color="cyan",
        alpha=0.75,
        histtype="stepfilled",
        label="cyan",
    )

    x = data[f"phase_functions.Misc/Atlas.orange.H.value"] - data[f"sHG1G2_H_{f}"]
    ax[i, 2].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        color="orange",
        alpha=0.75,
        histtype="stepfilled",
        label="orange",
    )

# --------------------------------------------------------------------------------
# Atlas - ZTF colors
ySun = 5e3
for c in ["orange", "cyan"]:
    ax[0, 2].text(
        skatools.solar_color(f"Misc/Atlas.{c}", "Palomar/ZTF.g"),
        ySun,
        "$\\odot$",
        color=c,
        ha="center",
        va="center",
    )
    ax[1, 2].text(
        skatools.solar_color(f"Misc/Atlas.{c}", "Palomar/ZTF.r"),
        ySun,
        "$\\odot$",
        color=c,
        ha="center",
        va="center",
    )

# --------------------------------------------------------------------------------
# Axes
for a in ax.ravel():
    a.set_yscale("log")


ax[1, 0].set_xlabel("$\Delta$H")
ax[1, 1].set_xlabel("$\Delta$H")
ax[1, 2].set_xlabel("$\Delta$H")

ax[0, 0].set_yticks([1, 10, 1e2, 1e3, 1e4])
ax[0, 0].set_yticklabels(["", "$10^1$", "$10^2$", "$10^3$", "$10^4$"])
ax[0, 0].set_xticks([-2, -1, 0, 1, 2])
ax[0, 0].set_xticklabels(["-2", "-1", "0", "1", ""])

ax[0, 0].set_ylabel("Count in g")
ax[1, 0].set_ylabel("Count in r")

ax[0, 0].text(
    0,
    1.4e4,
    "$\\texttt{HG}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)
ax[0, 1].text(
    0,
    1.4e4,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)
ax[0, 2].text(
    0,
    1.4e4,
    "Atlas - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)

ax[0, 2].legend(loc="lower right", fontsize="xx-small")

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.pgf"),
)
plt.close()